# Derivative Reduction by Adiabatic Gate
*Copyright (c) 2021 Institute for Quantum Computing, Baidu Inc. All Rights Reserved.*

## Outline
This tutorial will demonstrate how to calibrate an X gate using the DRAG (Derivative Reduction by Adiabatic Gate) technique using Quanlse. The outline of this tutorial is as follows:
- Introduction
- Preparation
- Define the waveform for DRAG
- Quanlse realization
- Summary

## Introduction

In superconducting circuits, one has to consider the leakage error due to the fact that superconducting circuits are not perfect two-level systems. For weakly anharmonic qubits, leakage into the third energy level takes the qubit out of the computational subspace. To overcome this issue, researchers proposed the DRAG procedure \[1\], which removes most of the leakage error by modifying the waveforms of the drive pulses.

## Preparation
After you have successfully installed Quanlse, you could run the Quanlse program below following this tutorial. To run this particular tutorial, you would need to import the following packages from Quanlse and other commonly-used Python libraries:

In [ ]:
# Import the Hamiltonian module
from Quanlse.QHamiltonian import QHamiltonian as QHam

# Import the function for calculating infidelity
from Quanlse.Utils.Infidelity import unitaryInfidelity

# Import related operators
from Quanlse.QOperator import driveX, driveY, driveZ, number, duff

# Import waveforms and functions used to process the waveforms' data
from Quanlse.QWaveform import gaussian, dragY1, QJob, QJobList, QWaveform

# Import simulator interface for Quanlse Cloud Service
from Quanlse.remoteSimulator import remoteSimulatorRunHamiltonian as runHamiltonian

# Import matplotlib for graphing purposes
import matplotlib.pyplot as plt

# Import numpy and scipy
import numpy as np
import math
from scipy import integrate

To use Quanlse Cloud Service, we need to acquire a token to get access to the cloud. 

In [ ]:
# Import Define class and set the token for cloud service
# Please visit http://quantum-hub.baidu.com
from Quanlse import Define
Define.hubToken = ""

## Define the waveform for DRAG

By performing a rotating wave approximation (RWA), the Hamiltonian in the rotating frame can be written as \[2\]: 

$$ 
\hat H_R / \hbar = \delta_1 |1\rangle \langle 1|+\delta_2 |2\rangle \langle 2|+\frac{\alpha_q}{2}\hat a^{\dagger}\hat a^{\dagger}\hat a \hat a+\frac{\varepsilon_x(t)}{2}
\left[ \hat{a}^\dagger + \hat{a} \right]+\frac{\varepsilon_y(t)}{2}
i \left[\hat{a}^\dagger - \hat{a}\right]
,
$$

where $\omega_1$ and $\omega_2$ are the qubits' frequencies; and $\omega_d$ is the driving frequency. $\alpha_q = \omega_2 -2\omega_1$ is the anharmonicity of the system. $\delta_1 = \omega_1-\omega_d$ and $\delta_2 = \omega_2-\omega_d$ are the detunings of the transitions with respect to the drive frequency. $\varepsilon_x(t)$ and $\varepsilon_y(t)$ are the pulses' amplitudes of the two independent quadrature controls (XY control).

In the ideal case, we can ignore the higher energy levels of the qubit. To implement a $\theta$ rotation about the x-axis, we set $\delta _1$ to be zero and solve the equation directly:
$$
\int_0^{t_g}\varepsilon_x(t)dt=\theta. 
$$

As for a Gaussian waveform $\varepsilon_G=Ae^{(t-\tau)^2 / 2\sigma^2}-B$, we solve $\int_0^{t_g}\varepsilon_G(t)dt=\theta$ to determine the amplitude $A$ corresponding to the $\theta$ rotation about the x-axis:
$$
A=\theta/\left( \int_0^{t_g}e^{-(t-\tau)^2/2\sigma^2}dt-t_ge^{-\tau^2/2\sigma^2} \right),
$$

$$
B=Ae^{-\tau^2/2\sigma^2}.
$$
In the equations above, $A$ ensures that the desired magnitude of rotation is implemented; while $B$ enforces that the pulse's amplitude start and end on zero. 

In the following code, we first define a couple of parameters to set the rotation angle and the anharmonicity term of the system. Then, we define the functions for calculating parameters of the Gaussian waveform (commonly used waveform functions are available in Quanlse).

In [ ]:
theta_x = np.pi # The angle of rotation
Delta = -0.4 * 2 * np.pi # The anharmonicity in GHz

# Calculate the parameters
def intTheta(tg):
    y = integrate.quad(gaussian(0, tg, 1, tg / 2, tg / 4), 0, tg)
    return y[0]

def calAx(tg):
    return theta_x / (intTheta(tg) - gaussian(0, tg, 1, tg / 2, tg / 4)(0) * tg)

def calBx(tg):
    return calAx(tg) * gaussian(0, tg, 1, tg / 2, tg / 4)(0)

In the DRAG procedure, the waveforms and detunings are modified to:
$$
\varepsilon_y(t) = -\frac{\dot {\varepsilon_x}(t)}{\alpha_q}, 
$$
$$
\delta_1(t) = -\frac{\varepsilon_x^2(t)}{2\alpha_q}.
$$

Here, we build the control pulses $\varepsilon_x(t)$ and $\varepsilon_y(t)$ and set the drive detuning $\delta_1$ according to the equations above.

In [ ]:
# Define the control waveforms
def epsilonX(t, params):
    tg = params['tg']
    a = calAx(tg)
    b = calBx(tg)
    return gaussian(0, tg, a, tg / 2, tg / 4)(t) - b
    
def epsilonY(t, params):
    tg = params['tg']
    a = calAx(tg)
    return dragY1(0, tg, a, tg / 2, tg / 4)(t) / Delta

# Set the drive detuning  
def delta1(t, params):
    tg = params['tg']
    lamda = np.sqrt(2)
    return - epsilonX(t, {"tg": tg}) ** 2 / 2 / Delta

## Quanlse realization

Quanlse stores the system's information required for simulation and optimization in the Hamiltonian. First of all, we create an empty Hamiltonian by instantiating an object from class `QHamiltonian`. In order to make a comparison to show the advantages of DRAG technique, we create two Hamiltonians: 'ham' will simulate without DRAG and 'hamDrag' with DRAG. 

In [ ]:
# Create empty Hamiltonians
ham = QHam(subSysNum=1, sysLevel=3, dt=0.2222)
hamDrag = QHam(subSysNum=1, sysLevel=3, dt=0.2222)

For this particular task, the system Hamiltonian can be expressed in four terms:
$$
\hat H_R = \hat H_{\rm drift} + \hat H_{\rm xctrl} + \hat H_{\rm yctrl}+ \hat H_{\rm freq} ,
$$
where $\hat H_{\rm drift}= \alpha_q\hat a^{\dagger}\hat a^{\dagger}\hat a \hat a/2$ represents the anharmonicity of the qubit, which is intrinsic and time-independent. We add the drift terms by calling `addDrift()`. The operator $\hat a^{\dagger}a^{\dagger} \hat a \hat a$ is defined as `duff()` in Quanlse, which takes the system's dimension as a parameter.

In [ ]:
# Add the anharmonic terms
ham.addDrift(duff(3), onSubSys=0, coef=Delta / 2.0)
hamDrag.addDrift(duff(3), onSubSys=0, coef=Delta / 2.0)

Next, the control terms $\hat H_{\rm xctrl}=(\hat a +\hat a^{\dagger})/2$, $\hat H_{\rm yctrl}=i(\hat a -\hat a^{\dagger})/2$ and $ \hat H_{\rm freq}=\hat a^{\dagger}\hat a $ are added by calling the function `addWave()`. Their according operators are also available and can be found in `QOperator`. Here we use the object of class `QJobList`, termed as 'Joblist', to add control wave uniformly. 
The code is as follows:

In [ ]:
# Gate times
t = np.arange(2., 9., 0.5)

# Intialize array index
jobList = ham.createJobList()
jobListDrag = hamDrag.createJobList()
for tg in t:
    jobWaves = jobList.createJob()
    jobWavesDrag = jobListDrag.createJob()
    # Add Gaussian Wave of X control on the qubit 0
    paraArgs = {"a": -0.5 * 2.0 * np.pi}
    # Add wave for the job list without DRAG pulses
    jobWaves.addWave(driveX(3), 0, QWaveform(epsilonX, 0, tg, {"tg": tg}))
    # Add wave for the job list with DRAG pulses
    jobWavesDrag.addWave(driveX(3), 0, QWaveform(epsilonX, 0, tg, {"tg": tg}))
    jobWavesDrag.addWave(driveY(3), 0, QWaveform(epsilonY, 0, tg, {"tg": tg}))
    jobWavesDrag.addWave(driveZ(3), 0, QWaveform(delta1, 0, tg, {"tg": tg}))
    # Append this job to the job list
    jobList.addJob(jobWaves)
    jobListDrag.addJob(jobWavesDrag)

For a thorough comparison, we compute gate fidelities within a range of gate times. In fact, the task can be done very efficiently using Quanlse. In particular, `runHamiltonian()` supports batch-job simulation and returns a list of dictionaries with details of the result, and the unitary operator is stored under the key `"unitary"`.

The simulation may take a long time to process on local devices. However, Quanlse provides a cloud service that could speed up this process significantly. To use Quanlse Cloud Service, the users can get a token from http://quantum-hub.baidu.com and use the functions in `remoteSimulator()` module to submit the job onto Quanlse's server. 

After the simulation, we assess the performance of the implemented gate using DRAG pulse by calculating the infidelity for various gate time defined as:

$$
{\rm infid} =1- \frac{1}{2}\left|{\rm Tr}(\hat{\sigma}_x P(U))\right|.
$$

Here, the projected evolution $P(U)$ ($U$ is the evolution of the system) in particular describes the evolution projected to the computational subspace consisting of the two lowest energy eigenstates $|0\rangle$ and $|1\rangle$; $\hat{\sigma}_x$ is the target gate we want to implement. 

In [ ]:
# Create the arrays for storing gate infidelities
errorX = np.zeros(len(t))
errorXDrag = np.zeros(len(t))

# Submit the job lists to Quanlse Cloud Service
result = runHamiltonian(ham, jobList=jobList)
resultDrag = runHamiltonian(hamDrag, jobList=jobListDrag)
errorX = []
errorXDrag = []

for index in range(len(t)):
    errorX.append(unitaryInfidelity(np.array([[0, 1], [1, 0]], dtype=complex), result[index]["unitary"], 1))
    errorXDrag.append(unitaryInfidelity(np.array([[0, 1], [1, 0]], dtype=complex), resultDrag[index]["unitary"], 1))

Finally, we can analyze and visualize the results using the Matplotlib library.

In [ ]:
plt.semilogy(t, errorXDrag, label='With DRAG', marker='.')
plt.semilogy(t, errorX, label='Without DRAG', marker='.')

plt.xlabel('Gate Time (ns)')
plt.ylabel('Infidelity')
plt.title('X Gate')
plt.legend()
plt.show()

As demonstrated above, most of the leakage error is mitigated. The blue (DRAG optimized waveform) line illustrates that DRAG reduces the infidelity by orders of magnitude.

## Summary
This tutorial introduces the DRAG technique using Quanlse. After reading this tutorial, users are encouraged to explore other advanced research which are different from this tutorial.

## References
\[1\] [Motzoi, Felix, et al. "Simple pulses for elimination of leakage in weakly nonlinear qubits." *Physical review letters* 103.11 (2009): 110501.](https://link.aps.org/doi/10.1103/PhysRevLett.103.110501)

\[2\] [Krantz, Philip, et al. "A quantum engineer's guide to superconducting qubits." *Applied Physics Reviews* 6.2 (2019): 021318.](https://aip.scitation.org/doi/abs/10.1063/1.5089550)